In [71]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping


In [72]:
english_sentences = ["hello", "how are you", "good morning", "good night", "thank you"]
french_sentences = ["bonjour", "comment vas-tu", "bonjour", "bonne nuit", "merci"]

In [73]:
french_sentences = ["starttoken " + sentence + " endtoken" for sentence in french_sentences]


In [74]:
french_sentences

['starttoken bonjour endtoken',
 'starttoken comment vas-tu endtoken',
 'starttoken bonjour endtoken',
 'starttoken bonne nuit endtoken',
 'starttoken merci endtoken']

In [75]:
batch_size = 2
epochs = 100
latent_dim = 256

In [76]:
# Initialize the Tokenizer for both source and target languages
english_tokenizer = Tokenizer(char_level = False)
fra_tokenizer = Tokenizer(char_level = False)

# Fit the tokenizer on the sentences
english_tokenizer.fit_on_texts(english_sentences)
fra_tokenizer.fit_on_texts(french_sentences)

In [77]:
english_tokenizer.word_index

{'you': 1,
 'good': 2,
 'hello': 3,
 'how': 4,
 'are': 5,
 'morning': 6,
 'night': 7,
 'thank': 8}

In [78]:
fra_tokenizer.word_index

{'starttoken': 1,
 'endtoken': 2,
 'bonjour': 3,
 'comment': 4,
 'vas': 5,
 'tu': 6,
 'bonne': 7,
 'nuit': 8,
 'merci': 9}

In [79]:
# Convert the sentences into sequences of integers
encoder_input_data = english_tokenizer.texts_to_sequences(english_sentences)
decoder_input_data = fra_tokenizer.texts_to_sequences(french_sentences)

In [80]:
max_encoder_seq_length = max([len(seq) for seq in encoder_input_data])
max_decoder_seq_length = max([len(seq) for seq in decoder_input_data])
max_encoder_seq_length, max_decoder_seq_length

(3, 5)

In [81]:
encoder_input_data = pad_sequences(encoder_input_data,
                                   maxlen = max_encoder_seq_length,
                                   padding = "post")
decoder_input_data = pad_sequences(decoder_input_data,
                                   maxlen = max_decoder_seq_length,
                                   padding = "post")

In [82]:
encoder_input_data

array([[3, 0, 0],
       [4, 5, 1],
       [2, 6, 0],
       [2, 7, 0],
       [8, 1, 0]], dtype=int32)

In [83]:
decoder_input_data

array([[1, 3, 2, 0, 0],
       [1, 4, 5, 6, 2],
       [1, 3, 2, 0, 0],
       [1, 7, 8, 2, 0],
       [1, 9, 2, 0, 0]], dtype=int32)

In [84]:
# Prepare the target data (decoder output, shifted by one time step)
decoder_output_data =np.zeros_like(decoder_input_data)
decoder_output_data[:, 0:-1] = decoder_input_data[:, 1:]

In [85]:
# Define the vocabulary size (total unique tokens)
num_encoder_tokens = len(english_tokenizer.word_index) + 1
num_decoder_tokens = len(fra_tokenizer.word_index) + 1

In [86]:
num_encoder_tokens, num_decoder_tokens

(9, 10)

In [87]:
# Input layer models
encoder_inputs = Input(shape = (None, ))
decoder_inputs = Input(shape = (None, ))


In [88]:
# Encoder
encoder_embedding = Embedding(input_dim = num_encoder_tokens,
                              output_dim = latent_dim,
                              input_length = max_encoder_seq_length)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [89]:
encoder_outputs

<KerasTensor shape=(None, 256), dtype=float32, sparse=False, name=keras_tensor_68>

In [90]:
state_h

<KerasTensor shape=(None, 256), dtype=float32, sparse=False, name=keras_tensor_69>

In [91]:
# Decoder
decoder_embedding = Embedding(input_dim = num_decoder_tokens,
                              output_dim = latent_dim,
                              input_length = max_decoder_seq_length)(decoder_inputs)
decoder_lstm = LSTM(latent_dim,return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding,
                                                 initial_state = [state_h, state_c])

In [92]:
# Dense layer for generating predictions
decoder_dense = Dense(num_decoder_tokens, activation = "softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [93]:
# define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [94]:
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_21            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, None, 256)      │          2,304 │ input_layer_20[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_5 (Embedding)   │ (None, None, 256)      │          2,560 │ input_layer_21[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_12 (LSTM)            │ [(None, 256), (None,   │        525,312 │ embedding_4[0][0]      │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_13 (LSTM)            │ [(None, None, 256),    │        525,312 │ embedding_5[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm_12[0][1],         │
│                           │ 256)]                  │                │ lstm_12[0][2]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, None, 10)       │          2,570 │ lstm_13[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,058,058 (4.04 MB)

 Trainable params: 1,058,058 (4.04 MB)

 Non-trainable params: 0 (0.00 B)

In [95]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [96]:
# Train the model
model.fit([encoder_input_data, decoder_input_data],
          np.expand_dims(decoder_output_data, -1),
          batch_size = batch_size,
          epochs = epochs)
#

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.2225 - loss: 2.2877
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4775 - loss: 2.1499
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.4775 - loss: 1.9205
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4775 - loss: 1.5018
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5025 - loss: 1.2251
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4525 - loss: 1.3474
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5025 - loss: 1.0953
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5500 - loss: 1.0821
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6275 - loss: 1.1593
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7150 - loss: 0.8703
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6700 - loss: 0.8100
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6775 - lo

In [97]:
# Create tinference models for the encoder and decoder for prediction
# Encoder model
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

In [104]:
# Decoder model (for inference)
decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_lstm_inf = LSTM(latent_dim, return_sequences = True, return_state = True)
decoder_output_inf, state_h_inf, state_c_inf = decoder_lstm_inf(decoder_embedding,
                                                                 initial_state = [decoder_state_input_h, decoder_state_input_c])

decoder_output_inf = decoder_dense(decoder_output_inf)
decoder_model = Model([decoder_inputs, decoder_state_input_h, decoder_state_input_c],
                      [decoder_output_inf, state_h_inf, state_c_inf])

In [105]:
# Function to decode the sequence
def decode_sequence(input_seq):
    # Get the encoder states
    states_value = encoder_model.predict(input_seq)

    # Generate an initial target sequence (the start token)
    target_seq = np.zeros((1,1))
    target_seq[0,0] = fra_tokenizer.word_index['starttoken'] # start token index

    # Sample output tokens
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # Correct the prediction input to match expected inputs
        output_tokens, h, c = decoder_model.predict([target_seq, states_value[1], states_value[2]])

        # Sample the next token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = fra_tokenizer.index_word[sampled_token_index]

        decoded_sentence += ' ' + sampled_token

        # Stop if we hit the end token or reach max length
        if sampled_token == 'endtoken' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence
        target_seq = np.zeros((1,1))
        target_seq[0,0] = sampled_token_index

        # Update the states
        states_value = [h, c]

    return decoded_sentence

In [106]:
# Test the decoder with a sample sentence

input_seq = encoder_input_data[1:2]      # Sample input sentence

In [107]:
encoder_input_data

array([[3, 0, 0],
       [4, 5, 1],
       [2, 6, 0],
       [2, 7, 0],
       [8, 1, 0]], dtype=int32)

In [108]:
decoded_sentence = decode_sequence(input_seq)
print("Decoded Sentence:", decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
Decoded Sentence:  comment


In [103]:
new = 'good night'